In [1]:
from _shared import *

## Задача 1. Конструирование метрики

Допустим, есть следующие данные о покупках в онлайн-магазине: стоимость (R), себестоимость (C) и факт обращения в поддержку (S). За каждое обращение в поддержку в среднем тратим 110 рублей на оплату работы оператора и компенсацию неудобств покупателям. Хотим максимизировать прибыль. Какую метрику лучше выбрать?

В вариантах ниже приведены формулы, по которым будем вычислять значение для каждой покупки и затем использовать для проверки гипотезы о равенстве средних. Значения переменных R, C и S неотрицательные. S равно 0, если обращения в поддержку не было, иначе равно 1.

In [2]:
answer = 'M = R - С - 110 * S'

## Задача 2. Функция вычисления метрики

In [3]:
"""
Напишите функцию get_metric, которая вычисляет значения метрики из предыдущего задания.

Шаблон решения
"""

import pandas as pd


def get_metric(df, support_cost):
    """Вычисляет значения метрики.

    :param df (pd.DataFrame): таблица с данными о заказах со столбцами:
        - revenue - выручка
        - cost_price - себестоимость
        - support - обращение в поддержку
    :param support_cost (float): средняя стоимость одного обращения в поддержку
    :return (pd.Series): значения метрики
    """
    return df['revenue'] - df['cost_price'] - support_cost * df['support']

In [4]:
df = pd.DataFrame({
    'revenue': [1500, 1800, 2100],
    'cost_price': [1300, 1200, 1600],
    'support': [1, 0, 0],
})
support_cost = 110
metric = get_metric(df, support_cost)
# metric = pd.Series([90, 600, 500])
metric

0     90
1    600
2    500
dtype: int64

In [5]:
# Solution (is the same)
def get_metric(df, support_cost):
    return df['revenue'] - df['cost_price'] - support_cost * df['support']

## Задача 3. Пороговое значение

Определите пороговое значении стоимости обращения в поддержку, при небольшом отклонение от которого меняется результат проверки гипотезы. Другими словами нужно найти значение, при котором p-value совпадает с уровнем значимости и среднее в экспериментальной группе больше, чем в контрольной.

Проверяем гипотезу о равенстве средних для метрики, которую вычисляли в предыдущем задании. В качестве критерия используем тест Стьюдента. Уровень значимости равен 0.05.

Ответ округлите до целого значения.

Код для генерации данных эксперимента

In [6]:
import numpy as np
import pandas as pd

alpha = 0.05
sample_size = 1000

df_control = pd.DataFrame({
    'revenue': [int(np.sin(x / 12) * 600 + 1200) for x in range(sample_size)],
    'cost_price': [int(np.sin(x / 12) * 400 + 700) for x in range(sample_size)],
    'support': (np.arange(sample_size) < sample_size - 400).astype(int),
})
df_pilot = pd.DataFrame({
    'revenue': [int(np.sin(x / 11 + 1) * 650 + 1250) for x in range(sample_size)],
    'cost_price': [int(np.sin(x / 11 + 1) * 400 + 700) for x in range(sample_size)],
    'support': (np.arange(sample_size) < sample_size - 300).astype(int),
})

In [7]:
res = []

for sc in [x for x in range(0, 1100, 1)]:
    control_values = get_metric(df_control, sc)
    pilot_values = get_metric(df_pilot, sc)
    
    res.append((
        sc,
        control_values.mean(),
        pilot_values.mean(),
        stats.ttest_ind(control_values, pilot_values).pvalue
    ))

df_res = pd.DataFrame(res, columns=['support_cost', 'mean_control', 'mean_pilot', 'p_val'])
df_res.head()

,support_cost,mean_control,mean_pilot,p_val
0,0,502.482,553.571,1.352363e-12
1,1,501.882,552.871,1.495924e-12
2,2,501.282,552.171,1.655137e-12
3,3,500.682,551.471,1.831740e-12
4,4,500.082,550.771,2.027670e-12


In [8]:
df_res['mean_diff'] = df_res['mean_pilot'] - df_res['mean_control']
df_res['is_significant'] = df_res['p_val'] < alpha

df_res.head()

,support_cost,mean_control,mean_pilot,p_val,mean_diff,is_significant
0,0,502.482,553.571,1.352363e-12,51.089,True
1,1,501.882,552.871,1.495924e-12,50.989,True
2,2,501.282,552.171,1.655137e-12,50.889,True
3,3,500.682,551.471,1.831740e-12,50.789,True
4,4,500.082,550.771,2.027670e-12,50.689,True


In [9]:
threshold = (
    df_res
    .groupby('is_significant')['support_cost']
    .min()
    .loc[False] - 1
)

threshold

317

In [10]:
df_res.iloc[316:320, :]

,support_cost,mean_control,mean_pilot,p_val,mean_diff,is_significant
316,316,312.882,332.371,0.048424,19.489,True
317,317,312.282,331.671,0.049924,19.389,True
318,318,311.682,330.971,0.051459,19.289,False
319,319,311.082,330.271,0.053030,19.189,False


In [11]:
#Solution
def get_metric(df, support_cost):
    return df['revenue'] - df['cost_price'] - support_cost * df['support']

for support_cost in range(316, 319):
    control_metric = get_metric(df_control, support_cost)
    pilot_metric = get_metric(df_pilot, support_cost)
    delta = pilot_metric.mean() - control_metric.mean()
    pvalue = stats.ttest_ind(control_metric, pilot_metric).pvalue
    print(f'support_cost={support_cost} delta={delta:0.1f}, pvalue={pvalue:0.4f}')

# support_cost=316 delta=19.5, pvalue=0.0484
# support_cost=317 delta=19.4, pvalue=0.0499
# support_cost=318 delta=19.3, pvalue=0.0515

support_cost=316 delta=19.5, pvalue=0.0484
support_cost=317 delta=19.4, pvalue=0.0499
support_cost=318 delta=19.3, pvalue=0.0515
